In [1]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler
# %config Completer.use_jedi = False

In [2]:
import numpy as np
from pvlib import pvsystem, singlediode
import sys
module_path = "/Users/jmccarty/Data/221205_ipv_workbench/github/IPV_Workbench"
sys.path.insert(0, module_path)
from ipv_workbench.utilities import utils, time_utils
from ipv_workbench.translators import panelizer
from ipv_workbench.devices import devices
import time
import pandas as pd
from ipv_workbench.solver import simulations as ipv_sim
from ipv_workbench.translators import mapping_irradiance as ipv_irrad
from ipv_workbench.translators import module_mapping as ipv_mm
from ipv_workbench.solver import calculations as ipv_calc, compile_mp, single_module_mp, topology_solver
from ipv_workbench.utilities import utils, time_utils
mp = False
display_print = False

In [3]:
project_folder = "/Users/jmccarty/Desktop/bano_simulations"
pickle_file = "/Users/jmccarty/.anydesk/incoming/2023-02-27 12:32:38.730/DP_solar_glass_B1393_raw.pickle"
cdte_pickle = "/Users/jmccarty/.anydesk/incoming/2023-02-28 07:03:20.065/CP_light_grey_B1393_raw.pickle"
cell_technology = "C"
orientation = "P"
panelizer_object = panelizer.PanelizedObject(project_folder, "B1393", cdte_pickle)
panelizer_object.analysis_location = 'zurich'
panelizer_object.analysis_year = 2020
panelizer_object.set_tmy_data()

# setting the analysis_period
panelizer_object.hourly_resolution = 2
panelizer_object.set_analysis_period()

custom_device_data = pd.read_csv(panelizer_object.module_cell_data, index_col='scenario').loc[
    f"{cell_technology}{orientation}"].to_dict()
panelizer_object.cell = devices.Cell(custom_device_data)

In [30]:
for surface in panelizer_object.get_surfaces():
    timeseries = panelizer_object.all_hoy
    timeseries = np.arange(8330,8345,1)
    tmy_location = utils.tmy_location(panelizer_object.tmy_file)
    dbt = panelizer_object.tmy_dataframe['drybulb_C'].values[timeseries]
    psl = panelizer_object.tmy_dataframe['atmos_Pa'].values[timeseries]

    print(f"    ...surface {surface}")

    surface_start = time.time()

    # load radiance data
    rad_surface_key = panelizer_object.get_dict_instance([surface])['DETAILS']['radiance_surface_label']
    total_ill = ipv_irrad.load_irradiance_file(panelizer_object.RADIANCE_DIR, rad_surface_key, "total").values[
        timeseries]
    direct_ill = ipv_irrad.load_irradiance_file(panelizer_object.RADIANCE_DIR, rad_surface_key, "direct").values[
        timeseries]
    diffuse_ill = np.where(total_ill < direct_ill, direct_ill * 0.01, total_ill - direct_ill)

    grid_pts = ipv_irrad.load_grid_file(panelizer_object.RADIANCE_DIR, rad_surface_key)

    for string in panelizer_object.get_strings(surface):
        print("     -------------------")

        print(f"    Starting string {string}")
        string_start = time.time()

        string_dict = panelizer_object.get_dict_instance([surface, string])
        string_details = string_dict['DETAILS']

        modules = panelizer_object.get_modules(surface, string)

        base_parameters = utils.get_cec_data(string_details['cec_key'], file_path=panelizer_object.cec_data)
        custom_module_data = pd.read_csv(panelizer_object.module_cell_data, index_col='scenario').loc[
            string_details['module_type']].to_dict()

        module_template = string_details['module_type']
        cell_type = ipv_mm.get_cell_type(module_template[0])
        orientation = ipv_mm.get_orientation(module_template[1])
        map_file = [fp for fp in panelizer_object.map_files if f"{cell_type}_{orientation}" in fp][0]
        default_submodule_map, default_diode_map, default_subcell_map = utils.read_map_excel(map_file)

        for module_name in modules:
            module_start = time.time()
            module_dict = panelizer_object.get_dict_instance([surface, string, module_name])
            pv_cells_xyz_arr = np.array(panelizer_object.get_cells_xyz(surface, string, module_name))

            module_i_dict, module_v_dict, module_g_dict, module_params = panelizer.compile_system_single_core(module_dict,
                                                                                                            timeseries,
                                                                                                            tmy_location,
                                                                                                            dbt, psl,
                                                                                                            pv_cells_xyz_arr,
                                                                                                            grid_pts,
                                                                                                            direct_ill,
                                                                                                            diffuse_ill,
                                                                                                            base_parameters,
                                                                                                            custom_module_data,
                                                                                                            default_submodule_map,
                                                                                                            default_diode_map,
                                                                                                            default_subcell_map,
                                                                                                            cell_type)

            module_dict['CURVES']['Imod'] = module_i_dict
            module_dict['CURVES']['Vmod'] = module_v_dict
            module_dict['YIELD']["initial_simulation"]['irrad'] = module_g_dict 
            module_dict['PARAMETERS'] = module_params
            module_end = time.time()
            
            for hoy in timeseries:
                if module_g_dict[hoy]>0:
                    module_p = module_i_dict[hoy] * module_v_dict[hoy]
                    mpp_p = np.max(module_p)
                    if mpp_p/module_g_dict[hoy] > 0.19:
                        print(module_name,hoy, mpp_p/module_g_dict[hoy])
                        

    ...surface {1393;0;5}
     -------------------
    Starting string {1393;0;5;1}
     -------------------
    Starting string {1393;0;5;0}
    ...surface {1393;0;2}
     -------------------
    Starting string {1393;0;2;4}
     -------------------
    Starting string {1393;0;2;5}
     -------------------
    Starting string {1393;0;2;0}
     -------------------
    Starting string {1393;0;2;1}
     -------------------
    Starting string {1393;0;2;2}
     -------------------
    Starting string {1393;0;2;3}
    ...surface {1393;0;3}


IndexError: list index out of range

In [21]:
module_g_dict[hoy]

2.6

In [7]:
G_eff_ann, C_temp_ann_arr = panelizer.build_module_features(module_dict, timeseries, tmy_location, dbt, psl, pv_cells_xyz_arr,
                                                            grid_pts, direct_ill, diffuse_ill,
                                                            base_parameters, custom_module_data,
                                                            default_submodule_map, default_diode_map, default_subcell_map,
                                                            cell_type)

In [157]:
hoy_idx = 9
irradiance_hoy = G_eff_ann[hoy_idx]
temperature_hoy = C_temp_ann_arr[hoy_idx]
submodule_map = module_dict['MAPS']['SUBMODULES']
subdiode_map = module_dict['MAPS']['DIODES']
subcell_map = module_dict['MAPS']['SUBCELLS']
ivcurve_pnts=250
    # TODO break apart into constituent pieces

submodules = np.unique(submodule_map)
diodes = np.unique(subdiode_map)

# we are going to solve the IV curves for current given voltages.
# Since we are solving at the level of the cell we need to parse down from the module range

# Basic assumption here: Module IV-curve can be converted to a cell IV-curve by
# dividing the module voltage by the number of cells the subcell current is calculated by
# dividing currents by the number of subcells.

submodule_i = []
submodule_v = []
for submodule_key in submodules:
    submodule_mask = submodule_map == submodule_key
    submodule_colmask = submodule_mask.flatten()

    submodule_diode = np.array(subdiode_map)[:, submodule_colmask]
    submodule_subcell = np.array(subcell_map)[:, submodule_colmask]

    submodule_irrad = irradiance_hoy[:, submodule_colmask]
    submodule_temp = temperature_hoy[:, submodule_colmask]

    diode_i = []
    diode_v = []
    for diode_key in diodes:
        diode_mask = submodule_diode == diode_key
        diode_colmask = diode_mask.flatten()

        submodule_subdiode_subcell = submodule_subcell[:, diode_colmask]

        submodule_subdiode_irrad = submodule_irrad[:, diode_colmask]
        submodule_subdiode_temp = submodule_temp[:, diode_colmask]
        for subcell_key in np.unique(submodule_subdiode_subcell):
                submodule_subdiode_subcell_mask = submodule_subdiode_subcell == subcell_key
                submodule_subdiode_subcell_irrad = submodule_subdiode_irrad[submodule_subdiode_subcell_mask]
                submodule_subdiode_subcell_temp = submodule_subdiode_temp[submodule_subdiode_subcell_mask]

                # Icell, Vcell = simulations.solve_subcells(parameters,
                #                                           submodule_subdiode_subcell_irrad,
                #                                           submodule_subdiode_subcell_temp, ivcurve_pnts=ivcurve_pnts)
                # cells_i.append(Icell)
                # cells_v.append(Vcell)


In [164]:
i,v = ipv_sim.solve_cells(module_params,
                    np.mean(submodule_subdiode_irrad,axis=0).flatten(),
                    np.mean(submodule_subdiode_temp,axis=0).flatten())

(2, 250)

array([66.27333333, 66.27333333])

In [145]:
np.mean(submodule_subdiode_irrad,axis=0)

array([37.67, 37.67])

In [110]:
idx_h = 10
hoy = timeseries[idx_h]
module_p = module_i_dict[hoy] * module_v_dict[hoy]
mpp_p = np.max(module_p)
mpp_p / np.sum(G_eff_ann[idx_h] * module_dict['PARAMETERS']['one_subcell_area_m2'])

0.3022256509583856

In [22]:
module_dict['PARAMETERS']

{'Technology': 'CdTe',
 'Bifacial': 0.0,
 'STC': 385.344,
 'PTC': 358.5,
 'A_c': 2.48,
 'Length': nan,
 'Width': nan,
 'N_s': 264,
 'I_sc_ref': 2.49,
 'V_oc_ref': 214.3,
 'I_mp_ref': 2.23,
 'V_mp_ref': 172.8,
 'alpha_sc': 0.00137,
 'beta_oc': -0.60004,
 'T_NOCT': 50.4,
 'a_ref': 7.402658,
 'I_L_ref': 2.509123,
 'I_o_ref': 6.177725e-13,
 'R_s': 8.185414,
 'R_sh_ref': 1065.831543,
 'Adjust': -13.503751,
 'gamma_r': -0.261,
 'BIPV': 'N',
 'Version': 'SAM 2018.11.11 r2',
 'Date': '1/3/2019',
 'cell': 'cdte',
 'orientation': 'portrait',
 'cec_database_key': 'First_Solar__Inc__FS_6385',
 'Wp': 385,
 'cell_width': 4.5,
 'cell_height': 1884.0,
 'cell_col_spacing': 0.05,
 'cell_row_spacing': 0.0,
 'n_cols_ideal': 264,
 'n_rows_ideal': 1,
 'n_cols': 264,
 'n_rows': 3,
 'n_submodule': 1,
 'frame_size': 15.0,
 'module_width': 1231.15,
 'module_height': 1914.0,
 'N_s_ideal': 264,
 'N_p_ideal': 1,
 'N_p': 1,
 'total_cells': 792,
 'Nsubcell_col': 1.0,
 'Nsubcell_row': 3,
 'cell_area': 6.7200000000000

In [23]:
parameters = module_dict['PARAMETERS']
Geff = 1000
Tcell = 25

sde_args = pvsystem.calcparams_desoto(
        Geff,
        Tcell,
        alpha_sc=parameters['alpha_sc'],
        a_ref=parameters['a_ref'],
        I_L_ref=parameters['I_L_ref'],
        I_o_ref=parameters['I_o_ref'],
        R_sh_ref=parameters['R_sh_ref'],
        R_s=parameters['R_s']
    )
# sde_args has values:
# (photocurrent, saturation_current, resistance_series,
# resistance_shunt, nNsVth)

# Use Bishop's method to calculate points on the IV curve with V ranging
# from the reverse breakdown voltage to open circuit
kwargs = {
    'breakdown_factor': parameters['breakdown_factor'],
    'breakdown_exp': parameters['breakdown_exp'],
    'breakdown_voltage': parameters['breakdown_voltage'],
}
evaluated_voltages = utils.create_voltage_range(sde_args, kwargs, curve_pts=500)

# Basic assumption here: Module IV-curve can be converted to a cell IV-curve by
# dividing the module voltage by the number of cells
# the subcell current is calculated by dividing currents by the number of subcells.
I, V, P = singlediode.bishop88(evaluated_voltages, *sde_args, **kwargs)

i_sub = I / 1
v_sub = V / 264

In [24]:
P.max()

385.3388872969486

In [124]:
idx_h = 10
hoy = timeseries[idx_h]
module_p = i*v
mpp_p = np.max(module_p)
mpp_p / np.sum(G_eff_ann[idx_h] * module_dict['PARAMETERS']['one_subcell_area_m2'])

0.0018246295682545909